In [52]:
import numpy as np  
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical  
import matplotlib.pyplot as plt  
import math  
import cv2  
import scipy as sp
import pandas as pd
import random
import time
import matplotlib.pyplot as plt


In [53]:
def process_data():
    filename= "C:/Users/Komal/Desktop/classes/mlp/Kobe/data.csv"
    raw = pd.read_csv(filename)
    # drops = ['shot_id', 'team_id', 'team_name', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', \
             # 'matchup', 'game_event_id', 'game_id', 'game_date']
    # drops = ['team_id', 'team_name', 'matchup', 'game_id', 'game_date']
    drops = ['team_id', 'team_name', 'game_event_id', 'game_id', 'game_date', 'shot_id']
    for drop in drops:
        raw = raw.drop(drop, 1)
    # categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'opponent', 'season']
    # categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'opponent', 'season', 'shot_id', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', \
    #          'matchup', 'game_event_id', 'game_date']
    categorical_vars = ['action_type', 'combined_shot_type', 'shot_type', 'shot_zone_area', 'shot_zone_range', 'shot_zone_basic', 'opponent', 'season']
    for var in categorical_vars:
        raw = pd.concat([raw, pd.get_dummies(raw[var], prefix=var)], 1)
        raw = raw.drop(var, 1)
    for i, row in enumerate(raw.itertuples(), 1):
        if "@" in row.matchup:
            raw.set_value(row.Index, 'matchup', 0)
        else:
            raw.set_value(row.Index, 'matchup', 1)
    df = raw[pd.notnull(raw['shot_made_flag'])]
    indexOfNull = raw[raw['shot_made_flag'].isnull()].index.tolist()
    return raw, df, indexOfNull

def split_data(raw, df, indexOfNull):
    per_test =  round((15 * len(df))/100)
    n =[0 for i in range(per_test)]
    m = len(indexOfNull)-1
    for z in range(per_test):
        c = indexOfNull[m]+1
        x = raw.iloc[c]
        #print(pd.notnull(x['shot_made_flag']))
        flg = 0
        if(pd.isnull(x['shot_made_flag'])):
            #print(c)
            flg =1
            while flg==1:
                c = c+1
                x = raw.iloc[c]
                if pd.notnull(x['shot_made_flag']):
                    flg =0
        n[z] = c
        m=m-1;
    test_comp = raw.iloc[n]
    test = test_comp.drop('shot_made_flag', 1)
    test_y = test_comp['shot_made_flag']
    df = raw.drop(raw.index[n])
    df= df[pd.notnull(df['shot_made_flag'])]
    # separate df into explanatory and response variables
    train = df.drop('shot_made_flag', 1)
    train_y = df['shot_made_flag']
    return train, train_y, test, test_y

In [54]:
raw, df, indexOfNull = process_data()
x_train, y_train, x_test, y_test = split_data(raw, df, indexOfNull)
#print(x_train.shape[1])

C:\Users\Komal\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
C:\Users\Komal\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


146


In [55]:
def build_model():
    num_sensors=1
    TIME_PERIODS = 146
    num_classes = 1

    model_m = Sequential()

    model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(x_train.shape[1],)))

    model_m.add(Conv1D(200, 73, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
    model_m.add(Conv1D(90, 55, activation='relu'))
    model_m.add(MaxPooling1D(2,2))
    model_m.add(Dropout(0.25))

    model_m.add(Conv1D(300, 10, activation='relu'))
    model_m.add(Conv1D(110, 1, activation='relu'))
    model_m.add(MaxPooling1D(1,1))
    model_m.add(Dropout(0.25))

    model_m.add(Conv1D(500, 1, activation='relu'))
    model_m.add(Conv1D(130, 1, activation='relu'))
    model_m.add(MaxPooling1D(1,1))
    model_m.add(Dropout(0.25))

    model_m.add(Conv1D(400, 1, activation='relu'))
    model_m.add(Conv1D(120, 1, activation='relu'))
    model_m.add(MaxPooling1D(1,1))
    model_m.add(Dropout(0.25))

    model_m.add(GlobalAveragePooling1D())

    model_m.add(Dropout(0.5))

    model_m.add(Dense(num_classes, activation='sigmoid'))
    model_m.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model_m

In [56]:
def compute_and_save_model_weights(model_m):
    
    callbacks_list = [
    ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),EarlyStopping(monitor='acc', patience=1)
    ]
    BATCH_SIZE = 400
    EPOCHS = 1
    history = model_m.fit(x_train, y_train, batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.2, verbose=1)
    model_m.save_weights('my_model_weights.h5')

In [75]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.core import Reshape
from  keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical
import os


## builds vgg9 model architechture
model = build_model()

## checks if pre-calculated weights are stored in a file
exists = os.path.isfile('C:/Users/Komal/Desktop/classes/mlp/Kobe/my_model_weights.h5')
if not exists:
    compute_and_save_model_weights(model) ## If not stored then it computes new weights on the model that was built by training on train data

model.load_weights('my_model_weights.h5') ## loads the weights into the model architecture

model.pop() ## removes the fully connected layers
model.pop() 

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

bottleneck_features_train = model.predict(x_train)
print(np.shape(bottleneck_features_train))
np.savetxt('bottleneck_features_train.csv', bottleneck_features_train, delimiter=",")

(22443, 120)


In [76]:
filename= "C:/Users/Komal/Desktop/classes/mlp/Kobe/bottleneck_features_train.csv"
train = pd.read_csv(filename, header=None)
print(train)

       0    1    2    3    4    5         6         7    8         9   ...   \
0      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.070886  0.0  0.000000 ...    
1      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.154695  0.0  0.000000 ...    
2      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.271895  0.0  0.000000 ...    
3      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.069686  0.0  0.000000 ...    
4      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.067629  0.0  0.000000 ...    
5      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.110038  0.0  0.000000 ...    
6      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.181831  0.0  0.000000 ...    
7      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.108299  0.0  0.000000 ...    
8      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.332837  0.0  0.000000 ...    
9      0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.283659  0.0  0.000000 ...    
10     0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.179169  0.0  0.000000 ...    
11     0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.068